In [ ]:
# --- Cell 0: Select Input Data ---
%matplotlib widget

import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Reload modules
try:
    importlib.reload(opym.petakit)
    importlib.reload(opym.dataloader)
    importlib.reload(opym.viewer)
    importlib.reload(opym)
    print("Reloaded opym modules.")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")

# --- Create Interactive Widgets ---
print("\n1. Select the 'processed_tiff_series_split' directory to process.")
print("2. Click 'Set Input Directory'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (e.g., '.../processed_tiff_series_split'):</b>",
    show_only_dirs=True
)
load_button = widgets.Button(
    description="Set Input Directory", button_style="primary"
)
load_output = widgets.Output()

# --- Global variable to hold the selected path ---
source_processed_dir: Path | None = None

def on_load_button_clicked(b):
    global source_processed_dir
    with load_output:
        load_output.clear_output()
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return
        
        source_processed_dir = Path(dir_chooser.value)
        print(f"✅ Input directory set to: {source_processed_dir.name}")
        print("   You can now run Cell 1 to test parameters.")

load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, load_button, load_output]))

In [ ]:
# --- Cell 1: Interactive Parameter Tuner ---

import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Global var to store the path of the last run ---
last_output_path: Path | None = None

# --- Create Parameter Sliders ---
angle_slider = widgets.FloatSlider(
    value=21.72,
    min=20.0,
    max=25.0,
    step=0.01,
    description='Sheet Angle:',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
z_step_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.05,
    description='Z-Step (um):',
    readout_format='.2f',
    style={'description_width': 'initial'},
)

# --- Create Buttons and Output Areas ---
run_button = widgets.Button(description="Run Deskew & Rotate", button_style="success")
view_button = widgets.Button(description="View Last Run", button_style="info", disabled=True)

log_output = widgets.Output(layout={'border': '1px solid black', 'height': '200px', 'overflow_y': 'scroll'})
view_output = widgets.Output()

# --- Define Button Click Handlers ---
def on_run_button_clicked(b):
    global last_output_path
    
    # Disable buttons during processing
    run_button.disabled = True
    view_button.disabled = True
    
    with log_output:
        clear_output(wait=True)
        
        if source_processed_dir is None:
            print("❌ ERROR: Source directory not set. Please run Cell 0 first.")
            run_button.disabled = False
            return
            
        # 1. Get values from sliders
        angle = angle_slider.value
        z_step = z_step_slider.value
        
        # 2. Create unique directory name for this run
        # e.g., "DSR_a21.72_z1.00"
        dsr_dir_name = f"DSR_a{angle:.2f}_z{z_step:.2f}"
        ds_dir_name = f"DS_a{angle:.2f}_z{z_step:.2f}"
        
        # Store this path for the "View" button
        last_output_path = source_processed_dir / dsr_dir_name
        
        try:
            print(f"--- Running job with Angle={angle}, Z-Step={z_step} ---")
            print(f"Output will be in: {dsr_dir_name}")
            
            # 3. Call the processing function with these parameters
            opym.run_petakit_processing(
                source_processed_dir,
                ds_dir_name=ds_dir_name,
                dsr_dir_name=dsr_dir_name,
                sheet_angle_deg=angle,
                z_step_um=z_step,
            )
            
            print(f"\n✅ Job finished. Click 'View Last Run'.")
            
            # Enable the view button
            view_button.disabled = False
            
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
        finally:
            # Re-enable the run button
            run_button.disabled = False

def on_view_button_clicked(b):
    with view_output:
        clear_output(wait=True)
        view_button.disabled = True
        
        if last_output_path is None or not last_output_path.exists():
            print(f"❌ ERROR: Output directory not found: {last_output_path}")
            print("   Please run the 'Deskew & Rotate' job first.")
            view_button.disabled = False
            return
            
        try:
            print(f"--- Loading data from: {last_output_path.name} ---")
            
            # 1. Load data from the *specific* output path
            get_stack, t_max, z_max, c_max, y, x = opym.load_tiff_series(last_output_path)
            
            print(f"✅ Data loaded. Shape: T={t_max+1}, Z={z_max+1}, C={c_max+1}, Y={y}, X={x}")
            
            # 2. Launch viewers
            opym.single_channel_viewer(get_stack, t_max, z_max, c_max, y, x)
            opym.composite_viewer(get_stack, t_max, z_max, c_max, y, x)
            
        except Exception as e:
            print(f"\n❌ An unexpected error occurred while loading viewers: {e}", file=sys.stderr)
        finally:
            view_button.disabled = False

# --- Link buttons and display UI ---
run_button.on_click(on_run_button_clicked)
view_button.on_click(on_view_button_clicked)

param_box = widgets.VBox([angle_slider, z_step_slider])
button_box = widgets.HBox([run_button, view_button])

display(widgets.VBox([param_box, button_box, log_output, view_output]))